In [64]:
from os import walk
import json
import re
import statistics
from utils import data_analysis, pmi, pmi_to_dict, docs_distribution, new_distribution
from tqdm import tqdm

from Novelty import Newness, Uniqueness, Difference
from Surprise import Surprise
from utils import pmi, data_analysis, docs_distribution

In [66]:
from Divergences import Jensen_Shannon
import numpy as np
from Scoring import compute_scores

In [68]:
my_path = './Recipe_dataset/RecipeFullDataset/'
save_path = './Recipe_dataset/Recipe_with_scores/'
filenames = next(walk(my_path), (None, None, []))[2]

### First estimation for the Individual term divergence for newness

In [ ]:
thresholds_all = []
JS = Jensen_Shannon() 
means = []
std_devs = []
for i in tqdm(range(len(filenames))):
    file_recette = filenames[i]
    #Opening Json file with al lrecipes infos
    file_path = my_path + file_recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    #### COLLECTING ALL NECESSARY INFO
    KB_recettes, _ = data_analysis(recipe_dict)
    #print('Knowledge base size : ', len(KB_recettes))
    if len(KB_recettes) <= 0:
        continue


    recette_variations = KB_recettes
    KB_matrix,  KB_dist, Count_matrix = docs_distribution(KB_recettes, recette_variations)
    KB_size = list(range(KB_matrix.shape[0]))
    
    #Let's samples 15 recipe max per round -- we don't need more than that for threshold estimate
    lengths = min(len(KB_recettes), 50)
    for j in range(lengths):

        select_variation = KB_size + [len(KB_size)+j]
        NewKB_dist, variation_dist = new_distribution(Count_matrix, select_variation)
        
        KB_updated = KB_recettes + [KB_recettes[j]]

        JSD_vector = JS.linear_JSD(KB_dist, variation_dist) ## Here I get the individual divergence of each terms compare to the other -- now I want an average and a standard dev for that doc
        #appear = []
        #disappear = []
        #for k in range(len(JSD_vector)):
        #    if JSD_vector[k] != 0:
                #print(variation_dist[k] / (KB_dist[k]+1e-10))
         #       if variation_dist[k] / (KB_dist[k]+1e-10) > 0:
         #           if KB_dist[k] > 0:
         #               appear.append(variation_dist[k] / (KB_dist[k]))
         #       if KB_dist[k] / (variation_dist[k]+1e-10) > 0:
         #           if variation_dist[k] > 0:
         #               disappear.append(KB_dist[k] / (variation_dist[k]))
       # if len(appear) >1:
       #     avg_a = statistics.mean(appear)
       #     std_a = statistics.stdev(appear)
       # if len(disappear) >1:
       #     avg_d = statistics.mean(disappear)
        #    std_d = statistics.stdev(disappear)
        for i in range(len(JSD_vector)):
            if variation_dist[i] > KB_dist[i]:
                means.append(JSD_vector[i])
        
        #means.append(avg_a)
        #means.append(avg_d)
        #std_devs.append(std_a)
        #std_devs.append(std_d)
         


In [23]:
print('The average divergence of each terms for all documents in training is : ', statistics.mean(means))
print('The average standard deviation of the divergence of each terms for all documents in training is : ', statistics.stdev(means))

The average divergence of each terms for all documents in training is :  0.003309394000348933
The average standard deviation of the divergence of each terms for all documents in training is :  0.004848692406032415


In [79]:
thresholds_all = []
JS = Jensen_Shannon() 
means = []
std_devs = []
newness1s, newness2s, uniqueness1s, difference1s, new_surprise1s, dist_surprise1s, uniqueness2s, difference2s = [],[],[],[],[],[],[],[]
for i in tqdm(range(len(filenames))):
    file_recette = filenames[i]
    #Opening Json file with al lrecipes infos
    file_path = my_path + file_recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    #### COLLECTING ALL NECESSARY INFO
    KB_recettes, _ = data_analysis(recipe_dict)
    #print('Knowledge base size : ', len(KB_recettes))
    if len(KB_recettes) <= 0:
        continue

    lengths = min(len(KB_recettes), 50)
    if lengths >= 50:
        KB_recettes = KB_recettes[0:50]
        
    recette_variations = KB_recettes
    KB_matrix,  KB_dist, Count_matrix = docs_distribution(KB_recettes, recette_variations)
    KB_size = list(range(KB_matrix.shape[0]))
    neighbor_dist  = 0. 
    #Let's samples 15 recipe max per round -- we don't need more than that for threshold estimate
    KB_texts = ' '.join(KB_recettes).split()
    EB_PMI = pmi(KB_texts)
    dict_know_pmi = pmi_to_dict(EB_PMI)
    
    for j in range(lengths):

        select_variation = KB_size + [len(KB_size)+j]
        NewKB_dist, variation_dist = new_distribution(Count_matrix, select_variation)

        KB_updated = KB_recettes + [recette_variations[j]]
        updated_text = ' '.join(KB_updated).split()
        New_EB_PMI = pmi(updated_text)
        
        #return newness, novelty_new, uniqueness, novelty_uniq, diff_ratio, nolvety_diff, neighbor_dist, newratio_surprise_rate, newn_suprise, dist_surprise, uniq_surprise
        #newness1, _, uniqueness1, _, difference1, _, neighboroud_distance, new_surprise1, _, dist_surprise1, _ = compute_scores(KB_matrix, KB_dist, NewKB_dist, variation_dist, EB_PMI, dict_know_pmi, New_EB_PMI, neighbor_dist=neighboroud_distance)
        #newness2, _, uniqueness2, _, difference2, _, neighboroud_distance, _, _, _, _ = compute_scores(KB_matrix, KB_dist, NewKB_dist, variation_dist, EB_PMI, dict_know_pmi, New_EB_PMI, neighbor_dist=neighboroud_distance, newness_type='prob', uniq_type='shift', diff_type='global')   

        #newness = Newness(KB_dist, variation_dist)
        #newness1, _ = newness.divergent_terms(thr_div=0.0041, thr_new=0.0014)
        #newness2, _ = newness.probable_terms(thr_prob= 57.14, thr_new=0.0014)
    
        #uniqueness = Uniqueness(KB_dist)
        #uniqueness1, _ = uniqueness.dist_to_proto(variation_dist, thr_uniq=0.527)
        #uniqueness2, _ = uniqueness.proto_dist_shift(NewKB_dist, thr_uniqp=0.1295)
        
        #difference = Difference(KB_matrix, variation_dist, N=3)
        #if neighbor_dist==0.:
        #    neighbor_dist = difference.dist_estimate()
        #difference1, _ = difference.ratio_to_all(neighbor_dist, thr_diff=0.4177)
        #difference2, _ = difference.ratio_to_neighbors(neighbor_dist, thr_diff=0.1564)

        surprise = Surprise(New_EB_PMI)
        new_surprise1, _ = surprise.new_surprise(EB_PMI, thr_surp=0.0172)   ### Devrait être 0.0172
        dist_surprise1, _ = surprise.uniq_surprise(dict_know_pmi, eps= 0.00, thr_surp=0.036)   ### Devrait être 0.036

        #, newness_type='div', uniq_type='dist', diff_type='local',
        #newness1s.append(newness1)
        #newness2s.append(newness2)
        #uniqueness1s.append(uniqueness1)
        #uniqueness2s.append(uniqueness2)
        #difference1s.append(difference1)
        #difference2s.append(difference2)
        new_surprise1s.append(new_surprise1)
        dist_surprise1s.append(dist_surprise1)

 42%|█████████████████████████████████▌                                              | 210/501 [31:53<44:11,  9.11s/it]


KeyboardInterrupt: 

In [80]:
#print('The average newness 1 : ', statistics.mean(newness1s))
#print('The average standard deviation newness 1 : ', statistics.stdev(newness1s))
#print('=====================================================')
#print('The average newness 2 : ', statistics.mean(newness2s))
#print('The average standard deviation newness 2 : ', statistics.stdev(newness2s))
#print('=====================================================')
#print('The average uniqueness 1 : ', statistics.mean(uniqueness1s))
#print('The average standard deviation uniqueness 1 : ', statistics.stdev(uniqueness1s))
#print('=====================================================')
#print('The average uniqueness 2 : ', statistics.mean(uniqueness2s))
#print('The average standard deviation uniqueness 2 : ', statistics.stdev(uniqueness2s))
#print('=====================================================')
#print('The average difference 1 : ', statistics.mean(difference1s))
#print('The average standard deviation difference 1 : ', statistics.stdev(difference1s))
#print('=====================================================')
#print('The average difference 2 : ', statistics.mean(difference2s))
#print('The average standard deviation difference 2 : ', statistics.stdev(difference2s))
print('=====================================================')
print('The average new_surprise : ', statistics.mean(new_surprise1s))
print('The average standard deviation new_surprise : ', statistics.stdev(new_surprise1s))
print('=====================================================')
print('The average dist_surprise : ', statistics.mean(dist_surprise1s))
print('The average standard deviation dist_surprise : ', statistics.stdev(dist_surprise1s))

The average new_surprise :  0.0011671448974807505
The average standard deviation new_surprise :  0.0014953203307006618
The average dist_surprise :  0.0003664058656484847
The average standard deviation dist_surprise :  0.0004558997701943524
